In [1]:
import findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Домашнее задание 2") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
#Выберите 15 стран с наибольшим процентом переболевших на 31 марта 
#(в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

"""
Задание 1 - здесь вы вывели просто значение колонки total_cases - но эта колонка означает количество зафиксированных
на данный день случаев (суммарно), а не процент переболевших. Чтобы посчитать процент переболевших, нужно подумать,
из каких колонок это можно вывести. Самое простое - взять total_cases и поделить на population - уже будет неплохая оценка
процента переболевших. Только не забудьте умножить на 100.

"""



'\nЗадание 1 - здесь вы вывели просто значение колонки total_cases - но эта колонка означает количество зафиксированных\nна данный день случаев (суммарно), а не процент переболевших. Чтобы посчитать процент переболевших, нужно подумать,\nиз каких колонок это можно вывести. Самое простое - взять total_cases и поделить на population - уже будет неплохая оценка\nпроцента переболевших. Только не забудьте умножить на 100.\n\n'

In [5]:
from pyspark.sql.types import *
import pyspark.sql.functions as func

In [6]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("total_cases", IntegerType(), True),
                     StructField("population", IntegerType(), True)
                    ])

In [7]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [8]:
df = df.select('iso_code', 'location', 'total_cases', "population") \
.filter((df.date == '2020-03-31') & (df.continent.isNotNull()))


In [9]:
df = df.withColumn('Result', func.round(df['total_cases'] / df['population'] * 100,2))

In [10]:
df.sort(df.Result.desc()).show(15)

+--------+-------------+-----------+-----------+------+
|iso_code|     location|total_cases| population|Result|
+--------+-------------+-----------+-----------+------+
|     VAT|      Vatican|        6.0|      809.0|  0.74|
|     SMR|   San Marino|      236.0|    33938.0|   0.7|
|     AND|      Andorra|      376.0|    77265.0|  0.49|
|     LUX|   Luxembourg|     2178.0|   625976.0|  0.35|
|     ISL|      Iceland|     1135.0|   341250.0|  0.33|
|     ESP|        Spain|    95923.0|4.6754783E7|  0.21|
|     CHE|  Switzerland|    16605.0|  8654618.0|  0.19|
|     LIE|Liechtenstein|       68.0|    38137.0|  0.18|
|     ITA|        Italy|   105792.0|6.0461828E7|  0.17|
|     MCO|       Monaco|       52.0|    39244.0|  0.13|
|     AUT|      Austria|    10180.0|  9006400.0|  0.11|
|     BEL|      Belgium|    12775.0|1.1589616E7|  0.11|
|     NOR|       Norway|     4641.0|  5421242.0|  0.09|
|     DEU|      Germany|    71808.0|8.3783945E7|  0.09|
|     FRA|       France|    52278.0|6.8147687E7|

In [11]:
'''
Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном 
порядке по убыванию (в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

Задание 2 - здесь вы используете неправильную колонку для выполнения вычислений. 
Total_cases - это суммарное количество случаев, зафиксированных К данной дате. Оно всегда увеличивается, 
поэтому когда вы взяли max(total_cases) - у вас всегда максимум будет 31-го числа (самая поздняя дата). 
Чтобы вычислить правильно, в этом задании нужно использовать колонку new_cases - это количество новых случаев, 
зафиксированных В данную дату.
'''



'\nTop 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном \nпорядке по убыванию (в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)\n\nЗадание 2 - здесь вы используете неправильную колонку для выполнения вычислений. \nTotal_cases - это суммарное количество случаев, зафиксированных К данной дате. Оно всегда увеличивается, \nпоэтому когда вы взяли max(total_cases) - у вас всегда максимум будет 31-го числа (самая поздняя дата). \nЧтобы вычислить правильно, в этом задании нужно использовать колонку new_cases - это количество новых случаев, \nзафиксированных В данную дату.\n'

In [12]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("new_cases", IntegerType(), True)
                    ])

In [13]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [14]:
df.select('iso_code', 'location', 'new_cases', 'date') \
.filter((df.date >= '2020-03-24') &  (df.date <= '2020-03-31') & (df.continent.isNotNull())) \
.createOrReplaceTempView("hmmm")

In [15]:
spark.sql("Select t2.date, t1.* " +
          "from (select iso_code, location, max(new_cases) mx_new_cases " +
          "from hmmm " +
          "group by iso_code, location) t1 left join hmmm t2 on t1.iso_code = t2.iso_code and t1.mx_new_cases = t2.new_cases " +
          "order by t1.mx_new_cases desc").show(15)

+----------+--------+--------------+------------+
|      date|iso_code|      location|mx_new_cases|
+----------+--------+--------------+------------+
|2020-03-31|     USA| United States|     26314.0|
|2020-03-25|     ESP|         Spain|      9630.0|
|2020-03-31|     FRA|        France|      7629.0|
|2020-03-27|     DEU|       Germany|      6933.0|
|2020-03-26|     ITA|         Italy|      6203.0|
|2020-03-31|     GBR|United Kingdom|      4534.0|
|2020-03-30|     IRN|          Iran|      3186.0|
|2020-03-31|     TUR|        Turkey|      2704.0|
|2020-03-28|     BEL|       Belgium|      1850.0|
|2020-03-26|     AUT|       Austria|      1321.0|
|2020-03-27|     NLD|   Netherlands|      1179.0|
|2020-03-28|     CHE|   Switzerland|      1148.0|
|2020-03-31|     BRA|        Brazil|      1138.0|
|2020-03-31|     CAN|        Canada|      1129.0|
|2020-03-25|     ISR|        Israel|      1117.0|
+----------+--------+--------------+------------+
only showing top 15 rows



In [16]:
'''
Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, 
кол-во новых случаев сегодня, дельта)

Задание 3 - здесь вычисляете верно, но не за тот год - нужно за 2021. 
И еще можно использовать функцию lag - это то же самое, что lead с параметром -1
'''


'\nПосчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. \n(например: в россии вчера было 9150 , сегодня 8763, итог: -387) \n(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, \nкол-во новых случаев сегодня, дельта)\n\nЗадание 3 - здесь вычисляете верно, но не за тот год - нужно за 2021. \nИ еще можно использовать функцию lag - это то же самое, что lead с параметром -1\n'

In [17]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("new_cases", IntegerType(), True)
                    ])

In [18]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [19]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag 


In [20]:
windowSpec  = Window.partitionBy("iso_code").orderBy("date")

In [21]:
ddf = df.select('iso_code', 'location', 'date', 'new_cases') \
.filter((df.iso_code == 'RUS') & (df.date >= '2021-03-24') &  (df.date <= '2021-04-01')) \
.withColumn("lag",lag("new_cases", 1).over(windowSpec)).na.fill(value=0)

In [22]:
ddf.withColumn('Result', ( ddf['new_cases'] - ddf['lag'] ) ).show()

+--------+--------+----------+---------+------+------+
|iso_code|location|      date|new_cases|   lag|Result|
+--------+--------+----------+---------+------+------+
|     RUS|  Russia|2021-03-24|   8769.0|   0.0|8769.0|
|     RUS|  Russia|2021-03-25|   9128.0|8769.0| 359.0|
|     RUS|  Russia|2021-03-26|   9073.0|9128.0| -55.0|
|     RUS|  Russia|2021-03-27|   8783.0|9073.0|-290.0|
|     RUS|  Russia|2021-03-28|   8979.0|8783.0| 196.0|
|     RUS|  Russia|2021-03-29|   8589.0|8979.0|-390.0|
|     RUS|  Russia|2021-03-30|   8162.0|8589.0|-427.0|
|     RUS|  Russia|2021-03-31|   8156.0|8162.0|  -6.0|
|     RUS|  Russia|2021-04-01|   9057.0|8156.0| 901.0|
+--------+--------+----------+---------+------+------+

